(sec:combination)=
# 데이터 결합

In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
pd.options.display.max_columns = 20

In [3]:
np.random.seed(12345)

In [4]:
import matplotlib.pyplot as plt

plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 계층 인덱싱

**다중 인덱스 사용 시리즈**

In [5]:
data = pd.Series(np.random.uniform(size=9),
                 index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1    0.929616
   2    0.316376
   3    0.183919
b  1    0.204560
   3    0.567725
c  1    0.595545
   2    0.964515
d  2    0.653177
   3    0.748907
dtype: float64

In [6]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

**시리즈 인덱싱/슬라이싱**

- 0-레벨 인덱싱

In [7]:
data["b"]

1    0.204560
3    0.567725
dtype: float64

- 0-레벨 슬라싱

In [8]:
data["b":"c"]

b  1    0.204560
   3    0.567725
c  1    0.595545
   2    0.964515
dtype: float64

- 0-레벨 팬시 인덱싱

In [9]:
data.loc[["b", "d"]]

b  1    0.204560
   3    0.567725
d  2    0.653177
   3    0.748907
dtype: float64

- 0-레벨과 1-레벨 동시 인덱싱/슬라이싱: `loc` 속성과 쉼표로 구분되는 레벨 별 인덱싱/슬라이싱 적용

In [10]:
data.loc[:, 2]

a    0.316376
c    0.964515
d    0.653177
dtype: float64

**스택과 언스택**

언스택은 행 인덱스의 가장 깊은 라벨을 열 인덱스의 가장 깊은 라벨로 변환한다.

In [11]:
data.unstack()

,1,2,3
a,0.929616,0.316376,0.183919
b,0.204560,NaN,0.567725
c,0.595545,0.964515,NaN
d,NaN,0.653177,0.748907


스택은 열 인덱스의 가장 깊은 라벨을 행 인덱스의 가장 깊은 라벨로 변환한다.

In [12]:
data.unstack().stack()

a  1    0.929616
   2    0.316376
   3    0.183919
b  1    0.204560
   3    0.567725
c  1    0.595545
   2    0.964515
d  2    0.653177
   3    0.748907
dtype: float64

**다중 행/열 인덱스 사용 데이터프레임**

In [13]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[["a", "a", "b", "b"], [1, 2, 1, 2]],
                     columns=[["Ohio", "Ohio", "Colorado"],
                              ["Green", "Red", "Green"]])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

행과 열 인덱스의 각 레벨에 이름 지정

In [14]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["state", "color"]
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

행 인덱스는 2개의 레벨로 구성

In [15]:
frame.index.nlevels

2

기본 인덱싱은 열 인덱스의 0-레벨에 적용

In [16]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

### 다중 인덱스 레벨 교환과 인덱스 라벨 정렬

**레벨 교환**

- 레벨 이름 활용

In [17]:
frame.swaplevel("key1", "key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

- 정수 레벨 활용

In [18]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [19]:
frame.swaplevel(0, 1)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

**인덱스 라벨 정렬**

먼저 1-레벨의 라벨을 정렬한다.

In [20]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

레벨을 교환한다.

In [21]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 레벨 단위 그룹화

In [22]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

레벨과 축을 활용한 그룹화
- `level="color"`: 열 인덱스의 1-레벨 기준
- `axis="columns"`: 열에 사용된 라벨 단위로 그룹화

In [23]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [24]:
frame.groupby(level="color", axis="columns").sum()

C:\Users\latte\AppData\Local\Temp\ipykernel_21000\775557097.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level="color", axis="columns").sum()


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 인덱스 지정과 초기화

In [25]:
frame = pd.DataFrame({"a": range(7), "b": range(7, 0, -1),
                      "c": ["one", "one", "one", "two", "two",
                            "two", "two"],
                      "d": [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


* `set_index()` 메서드: 열의 특정 라벨을 이용하여 (멀티)인덱스를 지정한다.

In [26]:
frame2 = frame.set_index(["c", "d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

인덱스에 사용된 열을 그대로 둘 수도 있다.

In [27]:
frame.set_index(["c", "d"], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

* `reset_index()` 메서드: (멀티)인덱스를 열로 변환시키고 정수 인덱스를 지정

In [28]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


- `drop=True` 옵션: 인덱스로 사용된 라벨을 모두 삭제

In [29]:
frame2.reset_index(drop=True)

,a,b
0,0,7
1,1,6
2,2,5
3,3,4
4,4,3
5,5,2
6,6,1


## 데이터 결합

### 일대다/다대일 합병

In [30]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [31]:
df2 = pd.DataFrame({"key": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

df2

,key,data2
0,a,0
1,b,1
2,d,2


- `on="key"`: 열 라벨 `"key"`를 기준으로 합병. 두 데이터프레임에 모두 사용된 라벨만 대상으로 함.
    이유는 `how="inner"`가 기본값이기 때문.

| `key` | `data1` | `data2` | 경우의 수 |
| :---: | :---: | :---: | :---: |
| `b` | `0, 1, 6` | `1` | 3 |
| `a` | `2, 4, 5` | `0` | 3 |

In [32]:
pd.merge(df1, df2, on="key") # how="inner" 이 기본값

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


- `on=None` 기본값: 공동으로 사용된 열 라벨을 기준으로 합병. 따라서 여기서는 위 결과와 동일.

In [33]:
pd.merge(df1, df2) # on=None

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


참고: 합병 기준값으로 사용된 라벨의 순서는 특정되지 않는다.

- `how="outer"` 키워드 인자: 양쪽 데이터프레임에 사용된 모든 라벨을 기준값으로 사용

| `key` | `data1` | `data2` | 경우의 수 |
| :---: | :---: | :---: | :---: |
| `b` | `0, 1, 6` | `1` | 3 |
| `a` | `2, 4, 5` | `0` | 3 |
| `c` | `3` | `NA` | 1 |
| `d` | `NA` | `2` | 1 |

In [34]:
pd.merge(df1, df2, how="outer")

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,c,3,<NA>
7,d,<NA>,2


**공통 라벨이 없는 경우**

In [35]:
df3 = pd.DataFrame({"lkey": ["b", "b", "a", "c", "a", "a", "b"],
                    "data1": pd.Series(range(7), dtype="Int64")})

df3

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [36]:
df4 = pd.DataFrame({"rkey": ["a", "b", "d"],
                    "data2": pd.Series(range(3), dtype="Int64")})

df4

,rkey,data2
0,a,0
1,b,1
2,d,2


- `left_on`과 `right_on`: 합병 기준으로 사용될 열을 따로따로 지정

In [37]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


In [38]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


**다대다 합병**

In [39]:
df1 = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                    "data1": pd.Series(range(6), dtype="Int64")})

df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [40]:
df2 = pd.DataFrame({"key": ["a", "b", "a", "b", "d"],
                    "data2": pd.Series(range(5), dtype="Int64")})

df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


- `how="left"` 키워드 인자: 왼쪽 데이터프레임의 라벨만 사용

| `key` | `data1` | `data2` | 경우의 수 |
| :---: | :---: | :---: | :---: |
| `b` | `0, 1, 5` | `1, 3` | 6 |
| `a` | `2, 4` | `0, 2` | 4 |
| `c` | `3` | `NA` | 1 |

In [41]:
pd.merge(df1, df2, on="key", how="left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


- `how="inner"` 키워드 인자: 양쪽 데이터프레임에 공동으로 사용된 라벨만 사용

| `key` | `data1` | `data2` | 경우의 수 |
| :---: | :---: | :---: | :---: |
| `b` | `0, 1, 5` | `1, 3` | 6 |
| `a` | `2, 4` | `0, 2` | 4 |

In [42]:
pd.merge(df1, df2, how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,b,5,1
9,b,5,3


**다중키 합병**

In [43]:
left = pd.DataFrame({"key1": ["foo", "foo", "bar"],
                     "key2": ["one", "two", "one"],
                     "lval": pd.Series([1, 2, 3], dtype='Int64')})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [44]:
right = pd.DataFrame({"key1": ["foo", "foo", "bar", "bar"],
                      "key2": ["one", "one", "one", "two"],
                      "rval": pd.Series([4, 5, 6, 7], dtype='Int64')})
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


- `on=["key1", "key2"]` 키워드 인자: 양쪽 데이트프레임이 공동으로 사용된 두 개의 키를 기준으로 사용

| `(key1, key2)` | `lval` | `rval` | 경우의 수 |
| :---: | :---: | :---: | :---: |
| `(foo, one)` | `1` | `4, 5` | 2 |
| `(bar, one)` | `3` | `6` | 1 |

In [45]:
pd.merge(left, right, on=["key1", "key2"]) # how="inner"

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,bar,one,3,6


- `on=None` 기본값: 공동 키만 대상. 여기서는 위 결과와 동일.

In [46]:
pd.merge(left, right) # on=None

,key1,key2,lval,rval
0,foo,one,1,4
1,foo,one,1,5
2,bar,one,3,6


- `how=outer` 키워드 인자: 양쪽 데이트프레임이 공동으로 사용된 두 개의 키를 기준으로 사용

| `(key1, key2)` | `lval` | `rval` | 경우의 수 |
| :---: | :---: | :---: | :---: |
| `(foo, one)` | `1` | `4, 5` | 2 |
| `(foo, two)` | `2` | `NA` | 1 |
| `(bar, one)` | `3` | `6` | 1 |
| `(bar, two)` | `NA` | `7` | 1 |

In [47]:
pd.merge(left, right, on=["key1", "key2"], how="outer")

,key1,key2,lval,rval
0,bar,one,3,6
1,bar,two,<NA>,7
2,foo,one,1,4
3,foo,one,1,5
4,foo,two,2,<NA>


In [48]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [49]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


- `on=key1` 키워드 인자: `key1` 라벨 기분으로 합병. 공동으로 사용된 키는 구분을 위해 자동으로 
`_x`, `_y` 접미사 사용.

| `key1` | `(key2_x, lval)` | `(key2_y, rval)` | 경우의 수 |
| :---: | :---: | :---: | :---: |
| `foo` | `(one, 1), (two, 2)` | `(one, 4), (one, 5)` | 4 |
| `bar` | `(one, 3)` | `(one, 6), (one, 7)` | 2 |

In [50]:
pd.merge(left, right, on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


접미사를 지정할 수도 있다.

In [51]:
pd.merge(left, right, on="key1", suffixes=("_left", "_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 인덱스 기준 합병

In [52]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})

left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [53]:
right1 = pd.DataFrame({"group_val": [3.5, 7]}, index=["a", "b"])

right1

,group_val
a,3.5
b,7.0


- `left_on="key", right_index=True`: 왼쪽 데이터 프레임의 `"key"` 열과 오른쪽 프레임의 인덱스를 기준으로 합병
    `c`는 포함되지 않음에 주의.
    
| `key(인덱스)` | `value` | `group_val` | 경우의 수 |
| :---: | :---: | :---: | :---: |
| `a` | `0, 2, 4` | `3.5` | 3 |
| `b` | `1, 4` | `7.0` | 2 |

In [54]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


아래와 같이 해도 된다.

In [55]:
pd.merge(right1, left1, right_on="key", left_index=True)

,group_val,key,value
0,3.5,a,0
2,3.5,a,2
3,3.5,a,3
1,7.0,b,1
4,7.0,b,4


주의사항: 이전과는 달리 `left1`의 인덱스가 그대로 사용된다.
단지 순서가 결합 방식에 따라 다르게 지정될 뿐이다.

- `how="outer"`: `c` 값도 포함

| `key(인덱스)` | `value` | `group_val` | 경우의 수 |
| :---: | :---: | :---: | :---: |
| `a` | `0, 2, 4` | `3.5` | 3 |
| `b` | `1, 4` | `7.0` | 2 |
| `c` | `5` | `NA` | 1 |

In [56]:
pd.merge(left1, right1, left_on="key", right_index=True, how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


**다중 인덱스 기준 합병**

In [57]:
lefth = pd.DataFrame({"key1": ["Ohio", "Ohio", "Ohio",
                               "Nevada", "Nevada"],
                      "key2": [2000, 2001, 2002, 2001, 2002],
                      "data": pd.Series(range(5), dtype="Int64")})

lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [58]:
righth_index = pd.MultiIndex.from_arrays(
    [
        ["Nevada", "Nevada", "Ohio", "Ohio", "Ohio", "Ohio"],
        [2001, 2000, 2000, 2000, 2001, 2002]
    ]
)

righth = pd.DataFrame({"event1": pd.Series([0, 2, 4, 6, 8, 10], dtype="Int64",
                                           index=righth_index),
                       "event2": pd.Series([1, 3, 5, 7, 9, 11], dtype="Int64",
                                           index=righth_index)})

righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

다중 인덱스를 기준으로 사용하려면 레벨 수에 해당하는 키 도는 다중 인덱스를 사용해야 한다.

| `(key1, key2) (다중 인덱스)` | `data` | `(event1, event2)` | 경우의 수 |
| :---: | :---: | :---: | :---: |
| `(Ohio, 2000)` | `0` | `(4, 5), (6, 7)` | 2 |
| `(Ohio, 2001)` | `1` | `(8, 9)` | 1 |
| `(Ohio, 2002)` | `2` | `(10, 11)` | 1 |
| `(Nevada, 2001)` | `3` | `(0, 1)` | 1 |

In [59]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


- `how="outer"` 키워드 인자

| `(key1, key2) (다중 인덱스)` | `data` | `(event1, event2)` | 경우의 수 |
| :---: | :---: | :---: | :---: |
| `(Ohio, 2000)` | `0` | `(4, 5), (6, 7)` | 2 |
| `(Ohio, 2001)` | `1` | `(8, 9)` | 1 |
| `(Ohio, 2002)` | `2` | `(10, 11)` | 1 |
| `(Nevada, 2001)` | `3` | `(0, 1)` | 1 |
| `(Nevada, 2002)` | `4` | `(NA, NA)` | 1 |
| `(Nevada, 2003)` | `NA` | `(2, 3)` | 1 |

In [60]:
pd.merge(lefth, righth, left_on=["key1", "key2"], right_index=True, how="outer")

,key1,key2,data,event1,event2
4,Nevada,2000,<NA>,2,3
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11


**인덱스만 활용해서 합병하기**

동일한 하나의 키를 이용하는 것과 동일하게 작동한다.

In [61]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=["a", "c", "e"],
                     columns=["Ohio", "Nevada"]).astype("Int64")

left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [62]:
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=["b", "c", "d", "e"],
                      columns=["Missouri", "Alabama"]).astype("Int64")

right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [63]:
pd.merge(left2, right2, how="outer", left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


**join() 메서드**

인덱스 기준으로 합병을 지원한다.

In [64]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [65]:
left2.join(right2) # how="left" 기본값

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
c,3,4,9,10
e,5,6,13,14


In [66]:
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [67]:
left2.join(right2, how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1,2,<NA>,<NA>
b,<NA>,<NA>,7,8
c,3,4,9,10
d,<NA>,<NA>,11,12
e,5,6,13,14


왼쪽 데이터프레임의 키와 오른쪽 데이터프레임의 인덱스를 기준으로 합병할 수 있다.

- `on=키` 키워드 인자 활용

In [68]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [69]:
right1

,group_val
a,3.5
b,7.0


- left1의 "key" 열과 left2의 인덱스 기준 합병

In [70]:
left1.join(right1, on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


데이터프레임으로 구성된 인자를 `join()` 메서드의 인자로 사용하면
`pd.concat(axis=1)` 함수처럼 작동한다.

In [71]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=["a", "c", "e", "f"],
                       columns=["New York", "Oregon"])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [72]:
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [73]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0


In [74]:
left2.join([right2, another], how="outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1,2,<NA>,<NA>,7.0,8.0
c,3,4,9,10,9.0,10.0
e,5,6,13,14,11.0,12.0
b,<NA>,<NA>,7,8,NaN,NaN
d,<NA>,<NA>,11,12,NaN,NaN
f,<NA>,<NA>,<NA>,<NA>,16.0,17.0


### 종/횡 결합

`pd.concat()` 함수는 `np.concatenate()`와 유사하게 작동한다.

In [75]:
arr1 = np.arange(12).reshape((3, 4))
arr1

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [76]:
arr2 = np.arange(10, 22).reshape((3, 4))
arr2

array([[10, 11, 12, 13],
       [14, 15, 16, 17],
       [18, 19, 20, 21]])

In [77]:
np.concatenate([arr1, arr2], axis=1)

array([[ 0,  1,  2,  3, 10, 11, 12, 13],
       [ 4,  5,  6,  7, 14, 15, 16, 17],
       [ 8,  9, 10, 11, 18, 19, 20, 21]])

**시리즈 종/횡 결합**

In [78]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s1

a    0
b    1
dtype: Int64

In [79]:
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s2

c    2
d    3
e    4
dtype: Int64

In [80]:
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")
s3

f    5
g    6
dtype: Int64

In [81]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

**시리즈 횡 결합**

In [82]:
pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [83]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: Int64

**

In [84]:
pd.concat([s1, s4], axis="columns")

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


- `join="inner"`: 공동 인덱스 라벨만 사용

In [85]:
pd.concat([s1, s4], axis="columns", join="inner")

,0,1
a,0,0
b,1,1


- `keys=["one", "two", "three"]`: 종으로 결합되는 시리즈 각각에 해당되는 인덱스 레벨을 추가하여 다중 인덱스 지정.

In [86]:
result = pd.concat([s1, s1, s3], keys=["one", "two", "three"])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: Int64

In [87]:
type(result)

pandas.core.series.Series

In [88]:
result.unstack()

,a,b,f,g
one,0,1,<NA>,<NA>
two,0,1,<NA>,<NA>
three,<NA>,<NA>,5,6


- `axis="columns", keys=["one", "two", "three"`: 횡으로 결합하는 경우 `keys`로 지정된 값은 열의 라벨로 사용.

In [89]:
pd.concat([s1, s2, s3], axis="columns", keys=["one", "two", "three"])

,one,two,three
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


**데이터프레임 종/횡 결합**

시리즈의 경우와 유사하게 작동한다.

In [90]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=["a", "b", "c"],
                   columns=["one", "two"])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [91]:
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=["a", "c"],
                   columns=["three", "four"])
df2

,three,four
a,5,6
c,7,8


In [92]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

- 사전 활용

In [93]:
pd.concat({"level1": df1, "level2": df2}, axis="columns")

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

- 인덱스 레벨 이름 지정

In [94]:
pd.concat([df1, df2], axis="columns", keys=["level1", "level2"],
          names=["upper", "lower"])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

**행 인덱스 무시하기**

행의 인덱스가 중요하지 않다면 합병 후에 정수 인덱스로 초기화할 수 있다.

In [95]:
df1 = pd.DataFrame(np.random.standard_normal((3, 4)),
                   columns=["a", "b", "c", "d"])
df1

,a,b,c,d
0,1.248804,0.774191,-0.319657,-0.624964
1,1.078814,0.544647,0.855588,1.343268
2,-0.267175,1.793095,-0.652929,-1.886837


In [96]:
df2 = pd.DataFrame(np.random.standard_normal((2, 3)),
                   columns=["b", "d", "a"])
df2

,b,d,a
0,1.059626,0.644448,-0.007799
1,-0.449204,2.448963,0.667226


- `ignore_index=True`: 합병 후 인덱스 초기화

In [97]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.248804,0.774191,-0.319657,-0.624964
1,1.078814,0.544647,0.855588,1.343268
2,-0.267175,1.793095,-0.652929,-1.886837
3,-0.007799,1.059626,NaN,0.644448
4,0.667226,-0.449204,NaN,2.448963


- `ignore_index=False` 기본값을 사용하면 기존 인덱스 그대로 사용

In [98]:
pd.concat([df1, df2]) # ignore_index=False

,a,b,c,d
0,1.248804,0.774191,-0.319657,-0.624964
1,1.078814,0.544647,0.855588,1.343268
2,-0.267175,1.793095,-0.652929,-1.886837
0,-0.007799,1.059626,NaN,0.644448
1,0.667226,-0.449204,NaN,2.448963


### 겹침 결합

일부 또는 전체가 겹치는 두 개의 시리즈 또는 데이터프레임을 하나의 데이터프레임으로 결합하는 기법을 다룬다.

**시리즈 겹침 결합**

In [99]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=["f", "e", "d", "c", "b", "a"])
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [100]:
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=["a", "b", "c", "d", "e", "g"])

b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
g    5.0
dtype: float64

- `a.combine_first(b)`: `a`에 포함된 결측치를 `b`에 동일한 곳에 위치한 값으로 대체. 또한 인덱스 정렬 실행. 
인덱스는 두 시리즈의 인덱스 라벨을 모두 사용.

In [101]:
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    NaN
g    5.0
dtype: float64

**데이터프레임 겹침 결합**

열 대 열 기준으로 겹침 결합 실행.

In [102]:
df1 = pd.DataFrame({"a": [1., np.nan, 5., np.nan],
                    "b": [np.nan, 2., np.nan, 6.],
                    "c": range(2, 18, 4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [103]:
df2 = pd.DataFrame({"a": [5., 4., np.nan, 3., 7.],
                    "b": [np.nan, 3., 4., 6., 8.]})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [104]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 모양 변환

### 항목 재배열

- 스택
- 언스택

In [105]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(["Ohio", "Colorado"], name="state"),
                    columns=pd.Index(["one", "two", "three"],
                    name="number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [106]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [107]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [108]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [109]:
result.unstack(level="state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [110]:
s1 = pd.Series([0, 1, 2, 3], index=["a", "b", "c", "d"], dtype="Int64")
s2 = pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
data2 = pd.concat([s1, s2], keys=["one", "two"])

data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [111]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [112]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

- `dropna=False` 키워드 인자: 항목 재배치 과정에서 발생하는 결측치를 그대로 둚

In [113]:
data2.unstack().stack(dropna=False)

C:\Users\latte\AppData\Local\Temp\ipykernel_21000\3936770077.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data2.unstack().stack(dropna=False)


one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

**레벨 활용 스택/언스택**

In [114]:
df = pd.DataFrame({"left": result, "right": result + 5},
                  columns=pd.Index(["left", "right"], name="side"))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [115]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [116]:
df.unstack(level="state").stack(level="side")

C:\Users\latte\AppData\Local\Temp\ipykernel_21000\2617337668.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.unstack(level="state").stack(level="side")


state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

**테이블 데이텃세에서 긴 모양 데이터프레임 생성**

예제: 거시경제 데이터셋 활용

- 데이터셋 다운로드

In [117]:
base_url = "https://raw.githubusercontent.com/codingalzi/datapy/master/jupyter-book/examples/"
file = "macrodata.csv"

data = pd.read_csv(base_url + file)

1959년부터 2009년까지 총 203개의 분기<font size='2'>quarter</font>별 데이터 샘플 포함

In [118]:
data.shape

(203, 14)

- `head()` 메서드: 처음 5행 확인. 열 라벨도 확인 가능

In [119]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


열 라벨은 거시경제 지표 14개

In [120]:
data.columns

Index(['year', 'quarter', 'realgdp', 'realcons', 'realinv', 'realgovt',
       'realdpi', 'cpi', 'm1', 'tbilrate', 'unemp', 'pop', 'infl', 'realint'],
      dtype='object')

열 일부 데이터만 사용

In [121]:
data = data.loc[:, ["year", "quarter", "realgdp", "infl", "unemp"]]
data.head()

,year,quarter,realgdp,infl,unemp
0,1959.0,1.0,2710.349,0.00,5.8
1,1959.0,2.0,2778.801,2.34,5.1
2,1959.0,3.0,2775.488,2.74,5.3
3,1959.0,4.0,2785.204,0.27,5.6
4,1960.0,1.0,2847.699,2.31,5.2


- 기간 인덱스: `pd.PeriodIndex` 객체 활용. 년도(`year`)와 분기(`quarter`) 결합

**주의사항**: `pop()` 메서드를 사용하기에 `data` 데이터프레임에서 해당 열 삭제됨

In [122]:
periods = pd.PeriodIndex(year=data.pop("year"),       # pop()
                         quarter=data.pop("quarter"), # pop()
                         name="date")                 # 인덱스 이름
periods

C:\Users\latte\AppData\Local\Temp\ipykernel_21000\3065240426.py:1: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  periods = pd.PeriodIndex(year=data.pop("year"),       # pop()


PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

- `to_timestamp("D")`: 일 단위의 `DatetimeIndex`로 변환

In [123]:
data.index = periods.to_timestamp("D")
data.index

DatetimeIndex(['1959-01-01', '1959-04-01', '1959-07-01', '1959-10-01',
               '1960-01-01', '1960-04-01', '1960-07-01', '1960-10-01',
               '1961-01-01', '1961-04-01',
               ...
               '2007-04-01', '2007-07-01', '2007-10-01', '2008-01-01',
               '2008-04-01', '2008-07-01', '2008-10-01', '2009-01-01',
               '2009-04-01', '2009-07-01'],
              dtype='datetime64[ns]', name='date', length=203, freq='QS-OCT')

In [124]:
data.head()

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


- 리인덱싱

In [125]:
data = data.reindex(columns=["realgdp", "infl", "unemp"])

열 인덱스에 이름 지정

In [126]:
data.columns.name = "item"

data.head()

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2


항목 재배열: 스택/언스택 활용

In [127]:
data.stack()

date        item   
1959-01-01  realgdp     2710.349
            infl           0.000
            unemp          5.800
1959-04-01  realgdp     2778.801
            infl           2.340
                         ...    
2009-04-01  infl           3.370
            unemp          9.200
2009-07-01  realgdp    12990.341
            infl           3.560
            unemp          9.600
Length: 609, dtype: float64

In [128]:
data.stack().reset_index()

,date,item,0
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
...,...,...,...
604,2009-04-01,infl,3.370
605,2009-04-01,unemp,9.200
606,2009-07-01,realgdp,12990.341
607,2009-07-01,infl,3.560


In [129]:
long_data = (data.stack()
             .reset_index()
             .rename(columns={0: "value"}))

In [130]:
long_data[:10]

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
5,1959-04-01,unemp,5.100
6,1959-07-01,realgdp,2775.488
7,1959-07-01,infl,2.740
8,1959-07-01,unemp,5.300
9,1959-10-01,realgdp,2785.204


### 피버팅

긴 모양 데이터프레임을 넓은 모양 데이터프레임으로 변환하는 과정을 살펴 본다.

**한 개의 열로 구성된 갑을 사용하는 피버팅**

In [131]:
pivoted = long_data.pivot(index="date", columns="item", values="value")

pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


**여러 개의 열로 구성된 갑을 사용하는 피버팅**

In [132]:
long_data["value2"] = np.random.standard_normal(len(long_data))
long_data[:10]

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,0.802926
1,1959-01-01,infl,0.000,0.575721
2,1959-01-01,unemp,5.800,1.381918
3,1959-04-01,realgdp,2778.801,0.000992
4,1959-04-01,infl,2.340,-0.143492
5,1959-04-01,unemp,5.100,-0.206282
6,1959-07-01,realgdp,2775.488,-0.222392
7,1959-07-01,infl,2.740,-1.682403
8,1959-07-01,unemp,5.300,1.811659
9,1959-10-01,realgdp,2785.204,-0.351305


값으로 사용되는 열 별로 열 인덱스 지정. 따라서 다중 인덱스가 열 인덱스로 사용됨.

In [133]:
pivoted = long_data.pivot(index="date", columns="item")
pivoted.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.575721  0.802926  1.381918
1959-04-01  2.34  2778.801   5.1 -0.143492  0.000992 -0.206282
1959-07-01  2.74  2775.488   5.3 -1.682403 -0.222392  1.811659
1959-10-01  0.27  2785.204   5.6  0.128317 -0.351305 -1.313554
1960-01-01  2.31  2847.699   5.2 -0.615939  0.498327  0.174072

In [134]:
pivoted["value"].head()

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2


피버팅은 `set_index()` 메서드와 `unstack()` 메서드를 연속적으로 적용하는 것과 동일한 결과를 낸다.

In [135]:
long_data.head(10)

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,0.802926
1,1959-01-01,infl,0.000,0.575721
2,1959-01-01,unemp,5.800,1.381918
3,1959-04-01,realgdp,2778.801,0.000992
4,1959-04-01,infl,2.340,-0.143492
5,1959-04-01,unemp,5.100,-0.206282
6,1959-07-01,realgdp,2775.488,-0.222392
7,1959-07-01,infl,2.740,-1.682403
8,1959-07-01,unemp,5.300,1.811659
9,1959-10-01,realgdp,2785.204,-0.351305


In [136]:
long_data.set_index(["date", "item"])

value    value2
date       item                        
1959-01-01 realgdp   2710.349  0.802926
           infl         0.000  0.575721
           unemp        5.800  1.381918
1959-04-01 realgdp   2778.801  0.000992
           infl         2.340 -0.143492
...                       ...       ...
2009-04-01 infl         3.370  0.069801
           unemp        9.200 -0.042890
2009-07-01 realgdp  12990.341  0.072754
           infl         3.560  0.436694
           unemp        9.600 -0.918250

[609 rows x 2 columns]

In [137]:
unstacked = long_data.set_index(["date", "item"]).unstack(level="item")
unstacked.head()

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-01-01  0.00  2710.349   5.8  0.575721  0.802926  1.381918
1959-04-01  2.34  2778.801   5.1 -0.143492  0.000992 -0.206282
1959-07-01  2.74  2775.488   5.3 -1.682403 -0.222392  1.811659
1959-10-01  0.27  2785.204   5.6  0.128317 -0.351305 -1.313554
1960-01-01  2.31  2847.699   5.2 -0.615939  0.498327  0.174072

### 언피버팅

넓은 모양 데이터프레임을 긴 모양 데이터프레임으로 변환하는 과정을 살펴 본다.

In [138]:
df = pd.DataFrame({"key": ["foo", "bar", "baz"],
                   "A": [1, 2, 3],
                   "B": [4, 5, 6],
                   "C": [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


- `set_index()` 메서드와 `stack()` 메서드를 연속적용하면 여러 개의 열이
하나의 열로 구성된 긴 모양 데이터프레임 생성 가능

In [139]:
df.set_index(["key"])

,A,B,C
key,,,
foo,1,4,7
bar,2,5,8
baz,3,6,9


In [140]:
df.set_index(["key"]).stack()

key   
foo  A    1
     B    4
     C    7
bar  A    2
     B    5
     C    8
baz  A    3
     B    6
     C    9
dtype: int64

In [141]:
reshaped = df.set_index(["key"]).stack().reset_index()
reshaped

,key,level_1,0
0,foo,A,1
1,foo,B,4
2,foo,C,7
3,bar,A,2
4,bar,B,5
5,bar,C,8
6,baz,A,3
7,baz,B,6
8,baz,C,9


In [142]:
reshaped.columns = ["key", "variable", "value"]
reshaped.sort_values("key")

,key,variable,value
3,bar,A,2
4,bar,B,5
5,bar,C,8
6,baz,A,3
7,baz,B,6
8,baz,C,9
0,foo,A,1
1,foo,B,4
2,foo,C,7


- `pd.melt()` 함수: 위 과정을 한 번에 처리한다.
    - `id_vars` 키워드 인자: 그룹 식별자 사용될 값들의 열 지정.

In [143]:
melted = pd.melt(df, id_vars="key")
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [144]:
melted = melted.sort_values("key")

melted

,key,variable,value
1,bar,A,2
4,bar,B,5
7,bar,C,8
2,baz,A,3
5,baz,B,6
8,baz,C,9
0,foo,A,1
3,foo,B,4
6,foo,C,7


- `pivot()` 메서드를 적용하면 다시 넓은 모양 데이터프레임으로 변환

In [145]:
reshaped = melted.pivot(index="key", columns="variable",
                        values="value")
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [146]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


- `value_vars=["A", "B"]` 키워드 인자: `value` 열에 사용되는 값과 연관된 열 지정. 여기서는 `A`, `B`열의 값만 사용.

In [147]:
pd.melt(df, id_vars="key", value_vars=["A", "B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


- `id_vars` 키워드 인자 생략: 그룹 식별자 사용하지 않음

In [148]:
pd.melt(df, value_vars=["A", "B", "C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [149]:
pd.melt(df, value_vars=["key", "A", "B"])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
